In [1]:
#문자열 전처리를 위한 모듈
#maketran
import string

#이미지벡터추출에서 쓰이는 모듈들
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as preprocess_input
from keras.models import Model

from PIL import Image
from keras.preprocessing import image as image

import numpy as np

#이미지벡터 데이터셋 파일로 저장하고 다시 불러올때, 쓰이는 모듈
import pickle
from pickle import load

#제너레이터 함수에서 쓰이는 모듈들
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from numpy import array

#glove 임베딩에서 쓰이는 모듈, 중복되서 주석화
#import numpy as np

#모델 정의에서 쓰이는 모듈들
from keras import Input, layers
from keras import optimizers

from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization

from keras.models import Model
from keras.layers.merge import add

'''
예전 tensorflow 하위 keras 모듈들
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM

'''

'\n예전 tensorflow 하위 keras 모듈들\nfrom tensorflow.keras.layers import Dropout\nfrom tensorflow.keras.layers import Dense\nfrom tensorflow.keras.layers import Embedding\nfrom tensorflow.keras.layers import LSTM\n\n'

In [20]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [2]:
'''
train은 이미 만들어서 잠시 주석처리
'''

#filename="C:/MyAI/archive/train_captions.txt"
filename="C:/MyAI/archive/train_captions.txt"


file = open(filename, 'r')
doc=file.read()

print(doc[0:300])

1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg,A girl going into a wooden building .
1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg,A little girl climbing the stairs to


In [3]:
'''
아래 실행시키거나 로드
'''

#6000개 train 데이터셋 딕셔너리화
train_descriptions = dict()
#key 값이 될 image id를 모아둘 dataset list
train_dataset=list()

i=0
for line in doc.split('\n'):
    tokens = line.split()
    if len(line)<2:
        continue

    #image_id 와 image_desc 분리
    tokens = line.split('.jpg,')
    
    #image_id,image_desc 만들기
    image_id, image_desc = tokens[0], tokens[1]

    if image_id not in train_descriptions:
        train_dataset.append(image_id)
        train_descriptions[image_id]=list()
        
        
    desc = 'startseq ' + ''.join(image_desc) + ' endseq'
  
    
    train_descriptions[image_id].append(desc)

    
#마지막 6000번째 이미지 이름에 대한 설명이 딕셔너리에 들어간지 확인.
print(train_dataset[-1])
print(train_descriptions[train_dataset[-1]])

3508882611_3947c0dbf5
['startseq A lady is running her dog through an agility course . endseq', 'startseq A large grey dog is jumping over a white hurdle . endseq', 'startseq A woman and a dog work through an agility course . endseq', 'startseq a woman running around a course with a grey dog jumping a railing endseq', 'startseq Trainer runs her dog through training course with green pipe endseq']


In [4]:
list(train_descriptions.keys())[:5]

['1000268201_693b08cb0e',
 '1001773457_577c3a7d70',
 '1002674143_1b742ab4b8',
 '1003163366_44323f5815',
 '1007129816_e794419615']

In [5]:
train_descriptions['1000268201_693b08cb0e']

['startseq A child in a pink dress is climbing up a set of stairs in an entry way . endseq',
 'startseq A girl going into a wooden building . endseq',
 'startseq A little girl climbing into a wooden playhouse . endseq',
 'startseq A little girl climbing the stairs to her playhouse . endseq',
 'startseq A little girl in a pink dress going into a wooden cabin . endseq']

In [6]:
#일단 6000개만 별도로 뺀걸 재확인
print(len(train_dataset))

#나중에 원하는 이미지만 찾아가기위한 path lisd 생성
train_image_path_list=list()
image_path=''

#문자열 합치는 과정을 위한 예시
image_path='C:/MyAI/archive/Images/' + train_dataset[i]+'.jpg'
print(image_path)

#train set image path list 생성하기
for i in range(len(train_dataset)):
    train_image_path_list.append('C:/MyAI/archive/Images/' + train_dataset[i]+'.jpg')

#test
print(len(train_image_path_list))

print(train_image_path_list[:5])

6000
C:/MyAI/archive/Images/1000268201_693b08cb0e.jpg
6000
['C:/MyAI/archive/Images/1000268201_693b08cb0e.jpg', 'C:/MyAI/archive/Images/1001773457_577c3a7d70.jpg', 'C:/MyAI/archive/Images/1002674143_1b742ab4b8.jpg', 'C:/MyAI/archive/Images/1003163366_44323f5815.jpg', 'C:/MyAI/archive/Images/1007129816_e794419615.jpg']


In [7]:
'''
여기는 하다보니 실행시켰는데 이 아래는 caption 데이터 전처리라 무의미
'''

# table은 딕셔너리형태이고, 첫번째 아규먼트는 key가 될 문자열, 두번째는 value가 될 문자열, 두개는 길이가 같아야한다.
# 세번째 아규먼트는 해당 value를 제외하고 싶은 key값import string
table = str.maketrans('', '', string.punctuation)

for key, desc_list in train_descriptions.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        # 리스트화(tokenize), 결과적으로 1문장이 스페이스 기준으로 한글자씩 리스트화
        desc = desc.split()
        # 소문자화
        desc = [word.lower() for word in desc]
        # 구두점 제거
        desc = [w.translate(table) for w in desc]
        # remove hanging 's' and 'a'
        desc = [word for word in desc if len(word)>1]
        # remove tokens with numbers in them
        desc = [word for word in desc if word.isalpha()]
        # store as string
        desc_list[i] =  ' '.join(desc)
        
#caption에 구두점 ""가 있는 이미지 997338199_7343367d7f 확인
train_descriptions['1000268201_693b08cb0e']

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq']

In [8]:
vocabulary = set()
for key in train_descriptions.keys():
    [vocabulary.update(d.split()) for d in train_descriptions[key]]
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 7667


In [9]:
# 파일에 description 저장
def save_descriptions(descriptions, filename):
        lines = list()
        for key, desc_list in descriptions.items():
            for desc in desc_list:
                lines.append(key + ' ' + desc)
        data = '\n'.join(lines)
        file = open(filename, 'w')
        file.write(data)
        file.close()

save_descriptions(train_descriptions, 'descriptions.txt')

In [10]:
#train caption 파일 불러오기
# 예제 코드 -> 대부분 함수로 선언해서 값 return 받는 형태
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [11]:
# photo identifiers 리스트 호출
def load_set(filename):
        doc = load_doc(filename)
        dataset = list()
        for line in doc.split('\n'):
            # 비어있는 라인 skip
            if len(line) < 1:
                continue

            identifier = line.split('.')[0]
            dataset.append(identifier)
        return set(dataset)

# train 데이터 불러오기
filename = "C:/MyAI/archive/train_captions.txt"
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 6000


In [12]:
# 빈출 어휘만 추출

all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)

# 10번 이상 언급된애만
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('preprocessed words %d ' % len(vocab))

preprocessed words 1667 


In [13]:
'''
중간 변수들 종합
0. train_descriptions ; image id 가 key 이고 caption들이 value 인 딕셔너리
1. train_dataset ; train_descriptions에서 key 값들이 모여있는 리스트
2. train_image_path_list ; 최초 다운받은 폴더의 Images 폴더에서 선별된 6000개 또는 2091개의 image만 찾아가기 위한 경로 모음(리스트)
3. vocabulary와 voca ; 전체 어휘와 빈출어휘
4. max_length ; 캡션 최대 길이
'''
print('train_descriptions : %d'% len(train_descriptions))
print('train_dataset: %d'% len(train_dataset))
print(train_dataset[0])
print('train_image_path_list: %d'% len(train_image_path_list))
print(train_image_path_list[0])
print('Original Vocabulary Size: %d' % len(vocabulary))
print('preprocessed words %d ' % len(vocab))
#print('Max CNT of Word of caption(description): %d' % max_length)

train_descriptions : 6000
train_dataset: 6000
1000268201_693b08cb0e
train_image_path_list: 6000
C:/MyAI/archive/Images/1000268201_693b08cb0e.jpg
Original Vocabulary Size: 7667
preprocessed words 1667 


In [14]:
#train caption set을 인덱싱화하자 generator function에 넣기 위해.
ixtoword = {}
wordtoix = {}
ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [15]:
vocab_size= len(ixtoword)+1
print(vocab_size)

1668


In [16]:
#필요변수인 Max Description length를 구하자 generator function에 넣기 위해.
# dictionary 에서 list로
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc
    
# clean된 caption들 중 가장 긴 문자은 단어가 몇개 일까?
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

# 제일 단어가 많이 쓰인 catpion에서 단어의 갯수
max_length = max_length(train_descriptions)
print('Max CNT of Word of caption(description): %d' % max_length)

Max CNT of Word of caption(description): 34


In [11]:
#8. Data Preprocessing — Images
'''
pkl파일로 이미 전처리 결과를 파일로 만들었으므로, 그냥 파일만 업로드하면됨.


관련 링크
https://keras.io/api/applications/inceptionv3/
https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3

Model 관련
https://www.tensorflow.org/api_docs/python/tf/keras/Model
https://ballentain.tistory.com/26
차원하는 이유 관련
https://github.com/pytorch/vision/blob/6db1569c89094cf23f3bc41f79275c45e9fcb3f3/torchvision/models/inception.py#L64
'''

'\n관련 링크\nhttps://keras.io/api/applications/inceptionv3/\nhttps://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3\n\nModel 관련\nhttps://www.tensorflow.org/api_docs/python/tf/keras/Model\nhttps://ballentain.tistory.com/26\n차원하는 이유 관련\nhttps://github.com/pytorch/vision/blob/6db1569c89094cf23f3bc41f79275c45e9fcb3f3/torchvision/models/inception.py#L64\n'

In [25]:
model = InceptionV3(weights='imagenet')
model_new = Model(model.input, model.layers[-2].output)

# example
image_path='C:/MyAI/archive/Images/3510695264_ef460fa6cc.jpg'
img = image.load_img(image_path, target_size=(299, 299))

print(img)

# PIL 이미지에 차원 추가 for inception_v3
x = image.img_to_array(img)

print(x.shape)

# PIL 이미지에 차원 추가 for inception_v3
x = np.expand_dims(x, axis=0)

print(x.shape)

# inception_v3에서 제공하는 preprocess_input()
x = preprocess_input(x)
print("After preprocess_input ",x.shape)

# model_new 활용
x=model_new(x)
print("After model_new ",x.shape)

# reshape from (1, 2048) to (2048, )
x = np.reshape(x, x.shape[1])
print(x.shape)
print(x)

<PIL.Image.Image image mode=RGB size=299x299 at 0x1A4F394C588>
(299, 299, 3)
(1, 299, 299, 3)
After preprocess_input  (1, 299, 299, 3)
After model_new  (1, 2048)
(2048,)
[0.5831281  0.19629233 0.36157233 ... 0.331434   0.98387724 0.29585987]


In [16]:
#image feature vector list 생성
image_feature_vector_list=[]
print(len(train_image_path_list))

'''
원래라면 위에서부터 train_blabla 말고 test_blabla라고 명명했어야하는데,
지금 짧은 시간 내에 일일이 바꾸기에는 촉박해서 그냥 변수명은 train_blabla인데,
내용물은 test 데이터셋입니다.
'''


for i in range(len(train_image_path_list)):
    train_image_path=train_image_path_list[i]
    img = image.load_img(train_image_path, target_size=(299, 299))
    vector_x=image.img_to_array(img)
    vector_x = np.expand_dims(vector_x, axis=0)
    vector_x = preprocess_input(vector_x)
    vector_x=model_new(vector_x)
    vector_x = np.reshape(vector_x, vector_x.shape[1])
    image_feature_vector_list.append(vector_x)

#test
print(len(image_feature_vector_list))
print(image_feature_vector_list[0])
print(image_feature_vector_list[0].shape)

2091
2091
[0.2336708  1.4837426  0.5093453  ... 0.61955357 0.8530511  0.6244722 ]
(2048,)


In [17]:
#train dataset의 원소는 3508882611_3947c0dbf5와 같이 확장자 없는 순수 이름을 형태로한 image_id
print(train_dataset[0])

'''
다행히 image_feature_vector_list의 순서는 train_dataset을 바탕으로 함. 즉 train_dataset의 n번째 값에 대응하는 image는
image_feature_vector_list의 n번째 값에 대응하는 image와 동일
결론적으로 n번째 이미지의 id는 train_dataset의 n번째, 벡터값은 image_feature_vector_list의 n번째에 위치.


다만 주어진 딕셔너리 photos의 key값은 위 train_dataset의 원소 +.jpg꼴이라 약간의 변형 필요.
'''

'''
원래라면 위에서부터 train_blabla 말고 test_blabla라고 명명했어야하는데,
지금 짧은 시간 내에 일일이 바꾸기에는 촉박해서 그냥 변수명은 train_blabla인데,
내용물은 test 데이터셋입니다.
'''


# train_dataset의 원소 +.jpg
key_photos=[]
for i in range(len(train_dataset)):
    key_photo=train_dataset[i]+'.jpg'
    key_photos.append(key_photo)

#딕셔너리 photos 만들기
photos=dict()

for i in range(len(train_dataset)):
    photos[key_photos[i]]=image_feature_vector_list[i]

#결과 일부 확인

i=0
for key, value in photos.items():
    if(i==10):
        break
    print(key, value)
    i=i+1
    
# Save the bottleneck test features to disk
with open("C:/MyAI/archive/encoded_test_images.pkl", "wb") as encoded_pickle:
    pickle.dump(photos, encoded_pickle)    


3509575615_653cbf01fc
3509575615_653cbf01fc.jpg [0.2336708  1.4837426  0.5093453  ... 0.61955357 0.8530511  0.6244722 ]
3509611207_7645b1d28d.jpg [0.48645875 0.2508113  1.2043017  ... 0.11045711 0.77078986 1.038936  ]
3510218982_318f738b76.jpg [0.68048835 0.62899965 0.27120814 ... 0.23872076 0.4604826  0.38286105]
3510219078_670b6b3157.jpg [0.28584427 0.34881198 0.15260641 ... 0.13671395 0.19284974 0.20186724]
3510695264_ef460fa6cc.jpg [0.5831281  0.19629233 0.36157233 ... 0.331434   0.98387724 0.29585987]
3511062827_cd87871c67.jpg [0.12558027 0.22470963 0.15222874 ... 0.9616679  0.15674192 0.3367479 ]
3511890331_6163612bb9.jpg [0.3843404  0.32294887 0.43583447 ... 0.41908535 0.54027665 0.1667529 ]
3512033659_7e8a0c2ffa.jpg [0.10077284 0.6326918  1.0875754  ... 0.12817284 0.49915388 0.27155975]
3512033861_a357bb58b6.jpg [0.08709924 0.8029965  0.01752065 ... 0.         0.03960565 0.40201607]
3512127856_18a4c7aace.jpg [0.4035572  0.18981698 0.26122856 ... 0.42452398 0.55984527 0.37261876

In [21]:
'''
이 부분이 저장한 pkl파일을 업로드하는 부분
자료형도 (2048,) 확인

'''

train_features = load(open("C:/MyAI/archive/encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

i=0
for key, value in train_features.items():
    if(i==10):
        break
    print(key, value)
    i=i+1
    
print(train_features['1000268201_693b08cb0e.jpg'].shape)

Photos: train=6000
1000268201_693b08cb0e.jpg [0.1227764  0.3329308  0.7527251  ... 0.2194151  0.30208534 0.4027964 ]
1001773457_577c3a7d70.jpg [0.73668337 0.591132   0.18194185 ... 0.16030258 0.3414473  0.9083165 ]
1002674143_1b742ab4b8.jpg [0.37349278 0.24600588 0.9633662  ... 1.1460401  0.2655153  0.01984664]
1003163366_44323f5815.jpg [0.48604354 0.29217845 0.31434855 ... 0.9825365  0.2810954  0.22330505]
1007129816_e794419615.jpg [0.21595949 0.15582629 0.29477617 ... 0.40562844 0.16532421 0.37413427]
1007320043_627395c3d8.jpg [0.17230766 0.49810642 0.21577148 ... 0.09679462 1.2298679  0.5421516 ]
1009434119_febe49276a.jpg [0.53430766 1.3675833  0.7805752  ... 0.39465535 0.8875862  1.072792  ]
1012212859_01547e3f17.jpg [0.24028873 1.0862249  0.04511875 ... 0.00142372 0.09225179 0.79243594]
1015118661_980735411b.jpg [0.23118024 0.12730137 0.39619404 ... 0.05207124 0.5935346  0.8125621 ]
1015584366_dfcec3c85a.jpg [0.4532365  0.4290843  0.2021637  ... 0.3568901  0.38221988 0.15984103]
(

In [32]:
# GENERATOR !!!!!!!!
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # input 시퀀스랑 output 시퀀스로 분리
                    in_seq, out_seq = seq[:i], seq[i]
                    # 인풋시퀀스 제로 패딩해서 길이 max로 맞추기
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # 아웃풋 시퀀스 인코딩(원핫 인코딩 추정), 1은 [1.0, 0, 0, blabla], 2는 [0, 1.0, 0, blabla]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
   
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)

            if n==num_photos_per_batch:
                return [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [22]:
# Load Glove vectors
glove_dir = 'C:/MyAI/archive'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [23]:
embedding_dim = 200
#voca_size란 max_length인줄 알았는데 아니고 전체 단어 갯수인듯
#근데 왜 선별된 1667이 아닌지는 몰겠음
vocab_size=1667+1

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
# 일단 문법그대로라면 np.zeros를 통해서 34행 200열의 영행렬이 생성
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [24]:
embedding_matrix.shape

(1668, 200)

In [25]:
# image feature extractor model
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# partial caption sequence model
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# decoder (feed forward) model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

# merge the two input models
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 34, 200)      333600      input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
______________________________________________________________________________________________

In [33]:
model.layers[2]
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

model.compile(loss='categorical_crossentropy', optimizer='adam')

epochs = 10
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('C:/MyAI/archive/model_' + str(i) + '.h5')

ValueError: in user code:

    C:\Users\schiz\anaconda3\envs\keras\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\schiz\anaconda3\envs\keras\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\schiz\anaconda3\envs\keras\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\schiz\anaconda3\envs\keras\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\schiz\anaconda3\envs\keras\lib\site-packages\tensorflow\python\keras\engine\training.py:541 train_step  **
        self.trainable_variables)
    C:\Users\schiz\anaconda3\envs\keras\lib\site-packages\tensorflow\python\keras\engine

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('C:/Users/User/Desktop/객체_플젝' + str(i) + '.h5')

In [ ]:
from keras.optimizers import Adam, RMSprop
#lr은 학습률, 예전에 이론 때 alpha라고 배운 변수명, 0~1사이의 값, 그 손실함수에서 극소값 찾아갈때 사용되는 변수, 이동 단위의 변수랄가
keras.optimizers.Adam(lr=0.001)
epochs = 10
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    # model.save('C:/Users/User/Desktop/객체_플젝' + str(i) + '.h5')

In [ ]:
model.save_weights('C:/Users/User/Desktop/객체_플젝30.h5')

In [ ]:
model.load_weights('C:/Users/User/Desktop/객체_플젝30.h5')

In [ ]:
with open("C:/Users/User/Desktop/객체_플젝/encoded_test_images.pkl", "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
pic_list=list()

pic_list.append('3510218982_318f738b76.jpg')
pic_list.append('3513362553_5fc5779e20.jpg')
pic_list.append('3516267455_ca17cc1323.jpg')
pic_list.append('3518675890_2f65e23ff9.jpg')
pic_list.append('3520321387_710ab74cda.jpg')

pic_list.append('3525417522_7beb617f8b.jpg')
pic_list.append('3535284878_f90f10236e.jpg')
pic_list.append('3537520829_aab733e16c.jpg')
pic_list.append('3538686658_30afc75f02.jpg')
pic_list.append('3540598210_972f0ff573.jpg')

In [ ]:
for z in range(0,10,1):
    #pic is image_id like '3516267455_ca17cc1323.jpg'
    #pic = list(encoding_test.keys())[z]
    pic=pic_list[z]
    #image is reshaped vector of image
    image = encoding_test[pic].reshape((1,2048))
    #images+pic is path of image that has image_id saved at pic
    x=plt.imread(images+pic)
    plt.imshow(x)
    plt.show()
    print("Greedy:",greedySearch(image))